# Linear SVM (Julia Implementation)

We are trying to solve a problem of binary classification. Lets assume we are given a set $S$ of $n \in \mathbb{N}$ datapoints $x_k \in \mathbb{R}^m$ with corresponding labels $y_k \in \{-1, 1\}$ (for our implementations it does not really matter whether the labels are given in that form as long as there are just binary labels with 1 beeing the positive case, but assuming this makes the derivation easier). Our goal is to build and train a **Support Vector Machine** (SVM) which can be used to predict labels of new datapoints. In contrast to most other methods we will try to achive this by using linear optimzation (Simplex Algorithm).  

For notation purposes we denote the set of all $x_k \in S$ with $y_k = 1$ as $S^+$ and the set of all $x_k = -1$ as $S^-$.

In [2]:
# Instalation and Setup
# import Pkg
# Pkg.add("JuMP")
# Pkg.add("GLPK")
using JuMP, GLPK
using Printf

### Finding one seperating hyperplane
The following algorithm aims to find any seperating hyperplane of the form $a^T x = \beta, a \in \mathbb{R}^m, \beta \in \mathbb{R}$ (with $a^T x_k \geq \beta$ if $x_k \in S^+$ and $a^T x_k \leq \beta$ if $x_k \in S^-$) to split the data into two half-spaces.  
**Why does it work?**: The Simplex Algorithm is only able to solve linear optimization problems with constraints that form a polyhedron. We can view the set of all hyperplanes which seperate our data as $H = \{(a, \beta) \in \mathbb{R}^{m+1} | a^T x - \beta \leq 0, \forall x \in S^+ \text{ and } a^T x - \beta \geq 0, \forall x \in S^- \}$ (we add a Restriction for every point in our provided dataset) which is clearly a polyhedron. Now optimizing over the linear objective function $0^T a + 0 \cdot \beta$ will provide one seperating hyperplane.

In [5]:
# Input: data dict of the form ["label": 1/0 or 1/-1 or..., "x": vector]
# Output: (a, beta) which seperate our data
function findHyperplane(data)
    model = Model(GLPK.Optimizer)

    set_optimizer_attribute(model, "msg_lev", GLPK.GLP_MSG_ALL)
    @variable(model, a[1:data[1].length])
    @variable(model, beta)
    for i in data
        # CHANGE: Change this if labels do not equal 1 fo good
        if i["label"] == 1
            @constraint(model, a' * i["x"] <= beta) 
         else
            @constraint(model, a' * i["x"] >= beta)
         end
    end
    @objective(model, Max, 0)
    JuMP.optimize!(model)
    return (JuMP.value.(a), JuMP.value.(beta))
end



1

### Findind a seperating hyperplane with maximal distance SVM
The following algorithm aims to find the seperating hyperplane with maximal distance (messured in the 1-Norm so we are able to linearize the problem) to points in the sets $S^+$ and $S^-$.  
**Why does it work?**: Our goal now is to maximize the distance between any point in $S^+$ or $S^-$ and our seperating hyperplane. We can solve this problem by creating two seperating hyperplanes given by the same vector $a$ but different levels $\beta_-$ and $\beta_+$. The level of the hyperplane that maximizes the distance (in the 1 norm) between points in the sets $S^+$ and $S^-$ is than given by $\frac{1}{2} \beta_- - \beta_+$. Adding the constraints $a^T x \leq \beta_+$, $a^T x \geq \beta_-$ and maximizing the distance between both hyperplanes $\beta_- - \beta_+$ achives this. It follows from the Hölder-Inequality that requiring $\|a\|_{\infty} = 1$ (or as a linear constraint $-1 \leq a_i \leq 1$) ensures actually maximizing the L1 distance between points in $S^+$ / $S^-$ and the seperating hyperplane. 

In [ ]:
# Input: data dict of the form ["label": 1/0 or 1/-1 or..., "x": vector]
# Output: (a, beta) which seperate our data optimally
function findHyperplane(data)
    model = Model(GLPK.Optimizer)

    set_optimizer_attribute(model, "msg_lev", GLPK.GLP_MSG_ALL)
    @variable(model, a[1:data[1].length])
    @variable(model, beta[1:2])
    for i in data
        # CHANGE: Change this if labels do not equal 1 fo good
        if i["label"] == 1
            @constraint(model, a' * i["x"] <= beta[1]) 
         else
            @constraint(model, a' * i["x"] >= beta[2])
         end
    end
    @constraint(model, -1 .<= a .<= 1)
    @objective(model, Max, 1/2 * (beta[2] - beta[1]))
    JuMP.optimize!(model)
    return (JuMP.value.(a), JuMP.value.(beta))
end